# AUA, DS 229 – MLOps
### Week 11 – SQL & SQLAlchemy

***

## What is Python `__name__`? (the question from the previous session)

In Python, `__name__` is a special variable that is automatically defined for every module. It is used to determine if a module is being run as the main program or if it is being imported as a module into another program.

When a Python script is executed, the interpreter sets the `__name__` variable to `"__main__"`. This indicates that the script is being run as the main program. On the other hand, if a module is imported into another program, its `__name__` variable is set to the module name.

This behavior is useful when writing Python scripts that can be used both as standalone programs and as modules that can be imported into other programs. By using the `__name__` variable, you can write code that is only executed when the script is run as the main program, but not when it is imported as a module.

In [ ]:
from dummy import get_name

In [ ]:
get_name()  # The script name is: `dummy.py`

In [ ]:
!python dummy.py

Why do we initialize Flask app with `__name__`?
```python
    # Inside app.py
    from flask import Flask
    app = Flask(__name__)
```
`__name__` is the name of the current Python module. The app needs to know where it’s located to set up some paths, and `__name__` is a convenient way to tell it that. Look through [this page](https://flask.palletsprojects.com/en/2.2.x/tutorial/factory/#:~:text=__name__%20is%20the,relative%20to%20the%20instance%20folder.) for more information.

Basically, `__name__` is used for Flask to configure where your application resides. If we set app name to some other string, then, in order to run the application we will need to execute `python app.py` drectly from the directory of `app.py`. Setting name to `__name__` makes it posible to run the script from outside.

## SQL

SQL (Structured Query Language) is a standard programming language used to manage and manipulate relational databases. It is used to create, modify, and query databases, and is considered one of the most popular and widely used database languages.

SQL is used to communicate with databases and perform operations like retrieving data, inserting data, updating data, and deleting data. It is used in a variety of applications, such as online banking, e-commerce, inventory management, and data analytics.

**SQL is a declarative language**, which means that users simply declare what they want to do with the data, and the database management system figures out how to do it. SQL syntax is relatively easy to learn and understand, and it is supported by many relational database management systems, including MySQL, Oracle, SQL Server, PostgreSQL, and SQLite.

SQL works under the hood by interacting with a database management system (DBMS). When a user sends an SQL query to the DBMS, the query is first parsed and checked for errors. If the query is valid, the **DBMS uses an optimizer to create an execution plan that determines the most efficient way to execute the query**. The execution plan involves several steps, including parsing the query, optimizing the plan, and executing the plan. The plan may involve accessing data from one or more tables, joining tables together, filtering data based on specified criteria, and returning the results to the user. When a query is executed, the DBMS may use various techniques to optimize performance, such as caching frequently accessed data in memory, creating indexes to speed up searches, and using parallel processing to execute multiple parts of a query at the same time.

> A **DBMS** is a software system that provides tools for managing and manipulating data stored in a database. It typically includes features for creating, modifying, and querying the database, as well as tools for managing users, security, backups, and other administrative tasks.  
> A **database engine**, on the other hand, is the core software component of a DBMS that is responsible for storing, retrieving, and managing data. It is the part of the DBMS that handles low-level data operations, such as reading and writing data to disk, indexing data for faster retrieval, and managing transactions to ensure data consistency.  
> While the terms DBMS and database engine can sometimes be used interchangeably, they can also be used to refer to different aspects of a database system. In general, the DBMS is the overall system that provides tools for managing and manipulating data, while the database engine is the software component that actually manages the data at a low level.

Here are some of the core keywords in SQL:

- **SELECT**: used to retrieve data from one or more tables in a database.  
- **FROM**: used to specify the table or tables from which to retrieve data.  
- **WHERE**: used to specify filtering criteria for selecting data.  
- **JOIN**: used to combine data from two or more tables based on a related column.  
- **INSERT**: used to add new data to a table.  
- **UPDATE**: used to modify existing data in a table.  
- **DELETE**: used to remove data from a table.  
- **CREATE**: used to create new tables, views, or other database objects.  
- **ALTER**: used to modify the structure of an existing database object, such as a table or view.  
- **DROP**: used to delete an existing database object.  
- **GROUP** BY: used to group data based on one or more columns.  
- **HAVING**: used to filter grouped data based on specified criteria.  
- **ORDER BY**: used to sort data based on one or more columns.  
- **DISTINCT**: used to retrieve unique values from a column.  
- **COUNT**: used to count the number of rows or non-null values in a column.  

### SQLite

<center><img src="./images/sqlite.png" width=500 height = 100/></center>

SQLite is a free, lightweight, and open-source relational database management system (RDBMS) that is widely used to manage structured data. It is a software library that provides a relational database management system, with support for SQL queries, transactions, and other features typically associated with a full-fledged RDBMS.

**SQLite is different from traditional client-server databases such as MySQL or PostgreSQL, as it is serverless and self-contained, meaning it does not require a separate server process and can be embedded directly into an application**. It is often used for small to medium-sized applications or for prototyping and testing purposes, due to its simplicity and ease of use.

## What do we have from the previous session?

In [ ]:
import pickle
import requests
import json
import random
from datetime import datetime, date, timedelta

import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [ ]:
# Reading the data.
cancer = load_breast_cancer()
features = ["mean concavity", "worst area", "mean area"]
df_feat = pd.DataFrame(cancer["data"], columns=cancer["feature_names"])
df_target = pd.DataFrame(cancer["target"], columns=["Cancer"])
df_feat = df_feat[features]

# Train / test split.
X_train, X_test, y_train, y_test = train_test_split(df_feat, np.ravel(df_target), 
                                                    test_size=0.20, random_state=5)

In [ ]:
# Data preprocessing.
scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.transform(X_test)

# Hyperparameter tuning.
param_grid = {"C": [0.1, 1, 10, 100, 1000], 
              "gamma": [1, 0.1, 0.01, 0.001, 0.0001], 
              "kernel": ["rbf", "linear", "poly"]}
model = GridSearchCV(SVC(), param_grid, refit=True, cv=3, verbose=1)
model.fit(X_train_transformed, y_train)

# Test set prformance.
pred = model.predict(X_test_transformed)
print(classification_report(y_test, pred))

### Create a database

- **Connection**: A connection represents a connection to a database. When you connect to a database, you create a connection object, which allows you to interact with the database. The connection object is responsible for managing transactions, executing SQL queries, and returning results.

- **Execute**: The `execute()` method is used to execute a SQL query. It takes a single string argument, which is the SQL query to execute. The result of the query (if any) can be accessed using the cursor object that was used to execute the query. The `execute()` method can also be used to execute parameterized queries, where query parameters are specified using placeholders in the SQL string.

- **Commit**: The `commit()` method is used to commit a transaction to the database. Transactions are used to group a series of database operations together into a single atomic operation. The `commit()` method is used to commit the changes made during a transaction to the database. If the transaction was successful, the changes are permanently saved to the database. If the transaction was not successful, the changes are rolled back (undone) and the database is left in its previous state.

It is important to note that transactions are not automatically committed. You must explicitly call the `commit()` method to commit a transaction. If you do not call `commit()`, the changes made during the transaction will be lost when the connection to the database is closed.


In [ ]:
import sqlite3

In [ ]:
DB_NAME = "cancer"
TABLE_NAME = "predictions"

In a relational database, a **primary key** is a unique identifier for a record in a table. It is a column or a combination of columns that uniquely identifies each row in the table. The primary key is used to enforce referential integrity and to ensure that each row in the table can be uniquely identified. A primary key must be unique, not null, and cannot be changed once it has been set.

A **foreign key**, on the other hand, is a field or combination of fields in a table that refers to the primary key of another table. It is used to establish a relationship between two tables in a relational database. A foreign key is a column in one table that refers to the primary key of another table. It is used to enforce referential integrity, which ensures that the data in related tables is consistent.

When a foreign key is created, a relationship is established between two tables. The table that contains the foreign key is called the referencing table or child table, and the table that contains the primary key is called the referenced table or parent table. The foreign key in the referencing table must have the same data type as the primary key in the referenced table. The foreign key can be null, which means that a row in the referencing table can exist without a corresponding row in the referenced table.


Here are the properties of primary key and foreign key in a relational database:

**Primary key**:

- Unique: A primary key must be unique and cannot contain duplicate values.
- Non-null: A primary key cannot contain null values. Every record in the table must have a primary key value.
- Unchangeable: Once a primary key value is set, it cannot be changed or updated.
- Single-valued: A primary key must contain a single value, i.e., it cannot be a combination of multiple columns.

**Foreign key**:

- Referential integrity: A foreign key ensures that the values in the referencing table (child table) match the values in the referenced table (parent table).
- Non-null: A foreign key can contain null values, but only if the referencing column is allowed to have nulls.
- Relationship: A foreign key establishes a relationship between two tables in a relational database.
- Data type: The data type of the foreign key column must match the data type of the primary key column in the referenced table.
- Cascade actions: Foreign keys can be configured to take actions when the referenced record is updated or deleted. For example, you can specify that all referencing records are also updated or deleted when the referenced record is updated or deleted. This is known as a cascading update or delete.

<center><img src="./images/primary_foreign_keys.png" width=900 height = 200/></center>

If a foreign key column in the referencing table contains a null value, it means that there is no corresponding record in the referenced table with the same primary key value. In other words, the relationship between the two tables is not established for that particular record. However, it's important to note that the use of null values in foreign keys can sometimes lead to data inconsistency or referential integrity issues, especially when a foreign key is part of a composite key (a combination of multiple columns). Therefore, it's generally recommended to avoid using null values in foreign keys, or to use them only when absolutely necessary and with careful consideration of the potential implications.

In [ ]:
conn = sqlite3.connect(f"{DB_NAME}.db")  # Creating a database.

columns = [
    "id INTEGER PRIMARY KEY",  # Primary key of our table.
    "country VARCHAR",
    "mean_concavity REAL",
    "worst_area REAL",
    "mean_area REAL",
    "predicted_class INTEGER", 
    "timestamp DATETIME"
]

create_table_cmd = f"CREATE TABLE {TABLE_NAME} ({','.join(columns)})"
conn.execute(create_table_cmd)  # Running SQL statement.
conn.commit()

In [ ]:
X_test.head()

Filling the data into table.

In [ ]:
def generate_rnd_dates(n):
    start, end = date(2019, 9, 1), date(2023, 6, 1)
    dates_diff = end - start
    total_days = dates_diff.days

    dates = []
    for idx in range(n):
        randay = random.randrange(total_days)
        dates.append(start + timedelta(days=randay))
    
    return dates
    

random_dates = generate_rnd_dates(len(X_test))
countries = ["AUT", "CZE", "NLD"]  # Randomly assigning one of these countries to each row.
random_country = lambda : random.choice(countries)
for idx in range(len(X_test)):
    mean_concavity, worst_area, mean_area = X_test.iloc[idx].values.tolist()
    country = random_country()
    date_time = str(random_dates[idx])
    row = f"{idx}, '{country}', {mean_concavity}, {worst_area}, {mean_area}, NULL, '{date_time}'"
    conn.execute(f"INSERT INTO {TABLE_NAME} VALUES ({row})")
    
conn.commit()
conn.close()

<div class="alert alert-block alert-danger">
<b>Action</b>:
    <b>Open a terminal an run</b>: `sqlite3 cancer.db`
</div> 

Now let's write some example queries for **predictions** table using Python.

In [ ]:
conn = sqlite3.connect(f"{DB_NAME}.db")
cur = conn.cursor()
cur.execute(f"SELECT * FROM {TABLE_NAME}")

data = cur.fetchmany(5)  # Use cur.fetchall() to read the whole data.
for row in data:
    print(row)
    
conn.close()

## SQLAlchemy

<center><img src="./images/sqlalchemy.jpeg" width=500 height = 100/></center>

[[image source](https://www.sqlalchemy.org/)]

SQLAlchemy is a popular Python library for working with relational databases. It provides a high-level, Pythonic interface for interacting with databases, while still allowing low-level access to the underlying SQL language. SQLAlchemy supports a wide range of relational databases, including MySQL, PostgreSQL, SQLite, Oracle, and Microsoft SQL Server.

Some of the key features of SQLAlchemy include:

- **ORM (Object-Relational Mapping) layer**: SQLAlchemy provides a powerful and flexible ORM layer that allows developers to interact with the database using Python objects, rather than writing raw SQL queries. This makes it easy to work with the database in a more object-oriented way, while still leveraging the full power of SQL.

<center><img src="./images/orm.png" width=500 height = 100/></center>

[[image source](https://gungadinn.github.io/data/2019/07/09/ORM/)]

- **SQL Expression Language**: SQLAlchemy also provides a SQL expression language that allows developers to construct SQL queries using Python code. This makes it easy to write complex queries in a more readable and maintainable way.

- **Connection Pooling**: SQLAlchemy supports connection pooling, which can improve the performance of database operations by reusing existing database connections, rather than creating new ones for each operation.

- **Database Abstraction**: SQLAlchemy provides a database-agnostic API, meaning that it abstracts away the differences between various database engines. This allows developers to write code that is portable across different databases.



SQLAlchemy supports two approaches to interacting with databases:

- **Core**: Provides a low-level SQL expression language and relational algebra API to interact with databases. It allows the user to build SQL queries and expressions using Python code.

- **ORM**: Provides a high-level Object-Relational Mapping API that allows the user to work with database data as Python objects. It maps Python classes to database tables and provides a way to query and manipulate database data using Python code.

In [ ]:
from sqlalchemy import create_engine, MetaData, inspect, text as SQL_text

In [ ]:
engine = create_engine(f"sqlite:///{DB_NAME}.db")

In [ ]:
# List the tables that form the cancer database.
metadata = MetaData()

inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Read the column information of 'predictions' table.
inspector.get_columns(TABLE_NAME)  # TABLE_NAME = 'predictions'.

In [ ]:
# Writing the first query with SQL.
with engine.connect() as connection:
    
    query = SQL_text(f"""
        SELECT * 
        FROM {TABLE_NAME}
    """)
    result = connection.execute(query)  # Returns a generator.
    
    # Returns a sequence of tuples (a returned tuple entries correspond to 
    # columns listed in SELECT clause)
    for row in result:
        print(row)
        
# No need to close the connection with `connection.close()` as the 
# context manager `engine.connect()` handles that part for us.      

In [ ]:
# Reading data with pandas.
data = pd.read_sql_query(sql=query, con=engine.connect())
data

In [ ]:
# A simple query example.
with engine.connect() as connection:
    
    query = SQL_text(f"""
        SELECT count(*), 
               count(DISTINCT country) 
        FROM {TABLE_NAME}
    """)
    result = connection.execute(query) 
    
    for row in result:
        print(row)

In [ ]:
# Explain what the following query computes:
with engine.connect() as connection:
    
    query = SQL_text(f"""
        SELECT country, 
               count(*) AS country_freq
        FROM {TABLE_NAME}
        WHERE timestamp BETWEEN '2021-01-01' AND '2022-12-31'
        GROUP BY country
        ORDER BY country_freq DESC
    """)
    result = connection.execute(query) 
    
    for row in result:
        print(row)
        

# Note:
# SQL 'BETWEEN' is inclusive, i.e. start and end values are included in the range.

### Object-Relational Mapping

ORM (Object-Relational Mapping) provides several advantages, including:

- **Simplified Database Access**: ORM provides a simplified way to interact with databases. Developers can use ORM frameworks to write SQL queries in an object-oriented way, which makes it easier to read, write, and maintain database code. This saves time and effort by reducing the amount of manual database manipulation code that developers need to write.

- **Platform Independence**: ORM provides a way to write database access code that is independent of the database management system (DBMS) being used. This means that developers can write code that works with any DBMS supported by the ORM framework, without having to worry about the specifics of the underlying database system.

- **Improved Code Quality**: ORM helps improve the quality of code by reducing the amount of boilerplate code that developers need to write. ORM frameworks provide many built-in features, such as caching, object validation, and data conversion, which can help reduce the amount of custom code that developers need to write. This makes the code more concise, easier to read, and less prone to errors.

- **Object-Oriented Programming**: ORM provides a way to use object-oriented programming techniques to interact with databases. This makes it easier to represent the database schema as a set of objects, which can help improve code readability and maintainability.

In [ ]:
# Let's drop our table and recreate it with ORM.
conn = sqlite3.connect(f"{DB_NAME}.db")
delete_table_cmd = f"DROP TABLE {TABLE_NAME}"
conn.execute(delete_table_cmd)
conn.commit()

In [ ]:
data.head()

#### Creating table with ORM

In [ ]:
# !pip install SQLAlchemy-Utils==0.40.0

In [ ]:
from sqlalchemy import Column, Integer, String, Float, TIMESTAMP, create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy_utils import create_database, database_exists

**The declarative base class `Base` works as a registry and maintains a catalog of classes and tables created with it**. There should normally be only one declarative base class in all commonly imported modules. This declarative base class should be inherited by all ORM classes. 

In [ ]:
Base = declarative_base()
metadata = Base.metadata


class CancerPredictions(Base):
    __tablename__ = "predictions"
    
    person_id = Column(Integer, primary_key=True)
    country = Column(String(100))
    mean_concavity = Column(Float)
    worst_area = Column(Float)
    mean_area = Column(Float)
    predicted_class = Column(Integer, nullable=True, default=None)  # nullable is False by default.
    timestamp = Column(TIMESTAMP, nullable=False)
    

We have just defined an ORM class CancerPredictions which is mapped to the **predictions** table. This class inherits the Base class so it is cataloged in the metadata of the Base class.

In [ ]:
# Let's recreate the database.
db_url = f"sqlite:///{DB_NAME}.db"

if not database_exists(db_url):
    create_database(db_url)  # Raises an exception if already exists.

The `db_url` is the same one that was used to create the engine. It is passed to the `create_database()` function from the SQLAlchemy-Utils library to create the target database. It should be noted that `create_database()` will raise an exception if the database to be created already exists, therefore we need to use the `database_exists()` function from the same library to check if the database already exists before creating it.

In [ ]:
# Create the defined table by using metadata.
engine = create_engine(db_url, pool_size=5, pool_recycle=3600)
metadata.create_all(bind=engine)

We can then use the **Base** class to create the **predictions** table for which an ORM class was defined above. Technically, the metadata of the Base class is used, and it will create all the tables whose ORM classes inherit this same Base class.

The `MetaData.create_all()` method creates all the tables stored in the metadata of the Base class. By default, tables already exist will be skipped and no exception will be raised.

> In SQLAlchemy, the `pool_size` parameter is used to specify the size of the connection pool. A connection pool is a cache of database connections that are kept open and ready to use, so that when an application requests a connection, it can be quickly obtained from the pool rather than having to establish a new connection every time.

> It's important to note that the optimal value for `pool_size` depends on various factors, such as the number of concurrent users, the available system resources, and the workload of the database. A higher pool size can improve performance for applications with high concurrency, but can also consume more resources. A lower pool size can conserve resources, but may lead to performance issues if there are too many concurrent users.

> The `pool_recycle` parameter in SQLAlchemy specifies the number of seconds after which a connection in the pool should be recycled, or reestablished with the database. This is useful because connections that remain open for too long can become stale and may cause errors when used. Setting `pool_recycle` too low can cause excessive connection churn and reduce performance, while setting it too high can allow stale connections to persist.

#### Inserting data and basic operations with tables

In [ ]:
from sqlalchemy.orm import sessionmaker, scoped_session

In [ ]:
engine = create_engine(db_url, pool_size=5, pool_recycle=3600)

# A session is an object that manages database transactions and 
# provides a high-level interface for working with a database.
session_factory = sessionmaker(bind=engine)
Session = scoped_session(session_factory)

**An ORM class is mapped to a table, and the instances of the class to the rows of the table**. Let’s create an instance of the CancerPredictions class and persist it to the database as a row.

In [ ]:
# We need to specify Python datetime objects for 'timestamp' column as 
# strings are not accepted.
str2datetime = lambda str_datetime: datetime.strptime(str_datetime, "%Y-%m-%d")

person = CancerPredictions(country="AUT", mean_concavity=0.04, worst_area=403.6, 
                           mean_area=637.2, timestamp=str2datetime("2022-02-17"))

# Adding a row to the table.
with Session() as sess:
    sess.add(person)
    sess.commit()

Notes:

- We didn’t create an `__init__()` method for the ORM class as it is created by SQLAlchemy under the hood.
- We can use the `Session` as a context manager, so we don’t need to explicitly close the Session when the DB operations are completed or failed.
- We need to add an ORM instance to the `Session` instance and then call the `commit()` method to persist it to the database. If you need to add multiple ORM instances, you can call the `add_all()` method of the `Session`.

In [ ]:
# Reading the newly-created record.
with Session() as sess:
    # Without first() it retrns a query object which is not readable.
    result = sess.query(CancerPredictions).filter(CancerPredictions.person_id == 1).first()
    
print(f"person_id = {result.person_id}")
print(f"country = {result.country}")
print(f"mean_concavity = {result.mean_concavity}")
print(f"worst_area = {result.worst_area}")
print(f"mean_area = {result.mean_area}")
print(f"predicted_class = {result.predicted_class}")
print(f"timestamp = {result.timestamp}")

The `first()` method returns the first result from the query. If there is no result, `None` will be returned. If we want to get all the results from the query, we can use the `all()` method, which will return a list of results, or an empty list if there is no result.

In [ ]:
with Session() as sess:
    result = sess.query(CancerPredictions).filter(CancerPredictions.person_id == 1).first()
    
    # Updating the value.
    result.worst_area = 408.7
    sess.add(result)
    sess.commit()
    
    result = sess.query(CancerPredictions).filter(CancerPredictions.person_id == 1).first()
    
print(f"worst_area = {result.worst_area}")

In [ ]:
with Session() as sess:
    result = sess.query(CancerPredictions).filter(CancerPredictions.person_id == 1).first()
    
    # Deleting the record.
    sess.delete(result)
    sess.commit()

#### Let's insert data into our table.

In [ ]:
data.head()

In [ ]:
data_list = []
labels = model.predict(X_test_transformed)
for idx, row in tqdm(enumerate(data.itertuples(index=False)), total=len(data)):
    new_record = CancerPredictions(person_id=row.id, country=row.country, 
                                   mean_concavity=row.mean_concavity,
                                   worst_area=row.worst_area, mean_area=row.mean_area, 
                                   predicted_class=int(labels[idx]), 
                                   timestamp=str2datetime(row.timestamp))
    data_list.append(new_record)

    
with Session() as sess: 
    sess.add_all(data_list)
    sess.commit()

In [ ]:
# Querying the data to make sure that is has been successfully inserted.
with Session() as sess:
    result = sess.query(CancerPredictions).all()
    
print(f"The number of queried rows: {len(result)}")
print(f"Type of each entry: {type(result[0])}")

Reading as pandas dataframe.

In [ ]:
session = Session()
people = session.query(CancerPredictions).all()  # A list of ORM objects.

df = pd.DataFrame([p.__dict__ for p in people])
df = df.drop("_sa_instance_state", axis=1)
df

**Note that the `__dict__` attribute is used to get the attributes of each ORM object as a dictionary**, and the **_sa_instance_state** attribute is dropped from the DataFrame because it is not relevant to the data.

> *_sa_instance_state* is an attribute of SQLAlchemy ORM objects that is used internally by SQLAlchemy to keep track of the state of the object.
When you create an ORM object, SQLAlchemy sets the *_sa_instance_state* attribute to an instance of the InstanceState class, which contains information about the object's state, such as whether it has been added to the session, whether its data has been modified, and so on.

Now let's reimplement this query with ORM:

```sql
        SELECT country, 
               count(*) AS country_freq
        FROM predictions
        WHERE timestamp BETWEEN '2021-01-01' AND '2022-12-31'
        GROUP BY country
        ORDER BY country_freq DESC
```

`func` is a module in SQLAlchemy that provides a way to call SQL functions in a database-agnostic way. This is useful because different databases have different SQL syntax for calling functions, and using `func` allows you to write SQL expressions that will work across different database backends.

In [ ]:
from sqlalchemy import func


with Session() as sess:
    results = (sess.query(CancerPredictions.country, func.count().label("country_freq"))
               .filter(CancerPredictions.timestamp.between("2021-01-01", "2022-12-31"))
               .group_by(CancerPredictions.country)
               .order_by(func.count().desc())
               .all())
    
results

### Deleting the database.

In [ ]:
# Dropping the table.
engine = create_engine(db_url, pool_size=5, pool_recycle=3600)
CancerPredictions.__table__.drop(engine)

In [ ]:
!rm -rf cancer.db

# References
- [SQLAlchemy](https://docs.sqlalchemy.org/en/20/)
- [What is an ORM, how does it work, and how should I use one?](https://stackoverflow.com/questions/1279613/what-is-an-orm-how-does-it-work-and-how-should-i-use-one)